In [1]:
import sys
sys.path.append('../')
from utility.file_utility import FileUtility
from bootstrapping.bootstrapping import BootStrapping
from make_representations.representation_maker import Metagenomic16SRepresentation
import pandas as pd

Using TensorFlow backend.
/home/nuria/.local/share/virtualenvs/MicroPheno-F70P6HaP/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nuria/.local/share/virtualenvs/MicroPheno-F70P6HaP/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nuria/.local/share/virtualenvs/MicroPheno-F70P6HaP/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np

# Crohn's Disease

### Ler fastaq

In [2]:
# LER OS FASTAQ

# fasta_files: arquivos fastaq
# mapping: orde na que foron lidos/iterados
fasta_files, mapping = FileUtility.read_fasta_directory('../../crohns_disease/samples_fastq/', 'fastq')
#fasta_files, mapping = FileUtility.read_fasta_directory('../../crohns_disease/samples_fastq/', 'fastq', 
#                only_files=["ERR1368879.fastq", "ERR1368880.fastq", "ERR1368881.fastq", 
#                            "ERR1368882.fastq", "ERR1368883.fastq", "ERR1368884.fastq",
#                            "ERR1368885.fastq", "ERR1368886.fastq", "ERR1368887.fastq",
#                            "ERR1368888.fastq", "ERR1368889.fastq", "ERR1368890.fastq"])

### 1. Bootstrapping

In [9]:
# CONSTRUIR OBXECTO PARA O BOOTSTRAPPING
'''
input_dir: input directory or a list of files
output_dir: a directory to generate the output files
sampling_sizes: a list of sampling sizes (N's)
n_resamples: number of resamples from each file (N_R = 10)
seqtype: file suffixes fastq or fasta etc.
M: number of files from the directory to make samplings from (M)
'''

BS=BootStrapping(fasta_files,'../../crohns_disease/results/bootstrapping/', seqtype='fastq',
                 sampling_sizes=[10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000], n_resamples=10, M=10)

In [ ]:
# add k-mer samplings
for k in [3,4,5,6,7,8]:
    print(k,'-mer bootstrapping..')
    BS.add_kmer_sampling(k)

In [19]:
# GARDAR RESULTADOS EN ARQUIVO 'bootstrapping_crohns_disease_prediction' NA RUTA QUE SE LLE INDICOU AO DEFINIR A INSTANCIA BS
BS.save_me('bootstrapping_crohns_disease_prediction')

### 2. k-mer representations

In [ ]:
# ENTRENAMENTO COS MELLORES TAMAÑOS DE K-MER (K) E OS SEUS CORRESPONDENTES TAMAÑOS DE MOSTRA (N) 
# (SEGÚN OS RESULTADOS DE BOOTSTRAPPING)

# dictionary of k-mers and their sampling values
# 6:[100,1000,2000,5000,10000,-1]  :  k-mer:[sampling values]      (-1 means using all sequences)
sampling_dict={3:[20],4:[100],5:[500],6:[100,1000,2000,5000,10000,-1],7:[5000],8:[8000]}
#sampling_dict={6:[5000]}

# number of cores
nump_p=20

# Para cada arquivo fastaq, creamos unha representación por cada posible combinación de tamaño de k-mer e tamaño de mostra, e gardamos os resultados
for k in sampling_dict.keys():  # for each k-mer
    for N in sampling_dict[k]:  # select a sampling size N
        print(k,'-mers with sampling size ',N)
        
        RS=Metagenomic16SRepresentation(fasta_files, mapping, N, nump_p)
        RS.generate_kmers_all(k, save='../../crohns_disease/datasets/dataset_'+str(k)+'-mers_rate_complete1359_seq_'+str(N))
        #RS.generate_kmers_all(k, save='../../crohns_disease/datasets/PRUEBA-dataset_'+str(k)+'-mers_rate_complete1359_seq_'+str(N))